In [18]:
import tensorflow as tf

In [19]:
sgd = tf.keras.optimizers.SGD(lr=0.0001,momentum=0.4)

In [20]:
bnmomemtum=0.4

#Define fire module
def fire(x, squeeze, expand):
    y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
    y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
    y1 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y1)
    y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
    y3 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y3)
    return tf.keras.layers.concatenate([y1, y3])

def fire_module(squeeze, expand):
    return lambda x: fire(x, squeeze, expand)

#Define SqueezeNet
x = tf.keras.layers.Input(shape=[224,224, 3]) # input is 192x192 pixels RGB

y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(64, 128)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.GlobalAveragePooling2D()(y)
y = tf.keras.layers.Dense(4, activation='softmax')(y)

model = tf.keras.Model(x, y)

model.compile(
  optimizer=sgd,
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])


In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = '/home/siddhant/Datasets/Minor/images/cropped/train/'
VAL_DIR = '/home/siddhant/Datasets/Minor/images/cropped/test/'
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen = ImageDataGenerator(rescale = 1./255,
                                 shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                                target_size = (224 , 224),
                                                batch_size = 16,
                                                class_mode = 'categorical')

val_set = val_datagen.flow_from_directory(VAL_DIR,
                                            target_size = (224 , 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')


Found 3400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [26]:
classweights = {0:1.0 , 1:1.1 , 2:1.1, 3:9.1}

In [27]:
from tensorflow.keras.callbacks import TensorBoard
tboard = TensorBoard(log_dir=f'log/')

In [28]:
training_history = model.fit_generator(training_set, epochs =20,validation_data = val_set , verbose = 1 ,steps_per_epoch=training_set.samples//training_set.batch_size, class_weight=classweights, callbacks=[tboard])

Epoch 1/20
212/212 [==============================] - 95s 448ms/step - loss: 2.0770 - acc: 0.4973 - val_loss: 0.8437 - val_acc: 0.7775
Epoch 2/20
212/212 [==============================] - 63s 296ms/step - loss: 1.3178 - acc: 0.7405 - val_loss: 0.7058 - val_acc: 0.8325
Epoch 3/20
212/212 [==============================] - 61s 288ms/step - loss: 1.0326 - acc: 0.8203 - val_loss: 0.5695 - val_acc: 0.9075
Epoch 4/20
212/212 [==============================] - 61s 287ms/step - loss: 0.8792 - acc: 0.8588 - val_loss: 0.5905 - val_acc: 0.8675
Epoch 5/20
212/212 [==============================] - 61s 287ms/step - loss: 0.7716 - acc: 0.8898 - val_loss: 0.5285 - val_acc: 0.8900
Epoch 6/20
212/212 [==============================] - 61s 289ms/step - loss: 0.6803 - acc: 0.9037 - val_loss: 0.3871 - val_acc: 0.9325
Epoch 7/20
212/212 [==============================] - 62s 295ms/step - loss: 0.6329 - acc: 0.9125 - val_loss: 0.4053 - val_acc: 0.9375
Epoch 8/20
212/212 [==============================] - 6

In [31]:
import pickle
with open('training_history', 'wb') as file_pi:
    pickle.dump(training_history.history, file_pi)